# 编程和测试要求
## 4. 多重加密

其实在建立加密类SAES的过程中我们已经实现了多重加密的功能，定义在类方法中

In [1]:
def _single_encrypt(self, plaintext):
    """单轮加密函数，输入明文，输出密文。加密key为16位"""

    # 密钥扩展
    keys = self.key_expansion()

    # 初始轮密钥加密
    initial_text = self.round_key_addition(plaintext, keys[0])

    # 第一轮操作
    after_sub_byte1 = self.sub_byte(initial_text)
    after_row_shift1 = self.row_shift(after_sub_byte1)
    after_mix_columns1 = self.mix_columns(after_row_shift1, self.MixMatrix)
    after_key_addition1 = self.round_key_addition(after_mix_columns1, keys[1])

    # 第二轮操作
    after_sub_byte2 = self.sub_byte(after_key_addition1)
    after_row_shift2 = self.row_shift(after_sub_byte2)
    ciphertext = self.round_key_addition(after_row_shift2, keys[2])

    return ciphertext

def _single_decrypt(self, ciphertext):
    """单轮解密函数，输入密文，输出明文。加密key为16位"""

    # 密钥扩展
    keys = self.key_expansion()

    # 初始轮密钥加密
    initial_text = self.round_key_addition(ciphertext, keys[2])

    # 第一轮逆操作
    after_inv_row_shift1 = self.row_shift(initial_text)
    after_inv_sub_byte1 = self.sub_byte(after_inv_row_shift1, use_inverse=True)
    after_key_addition1 = self.round_key_addition(after_inv_sub_byte1, keys[1])

    # 第二轮逆操作
    after_inv_mix_columns = self.mix_columns(after_key_addition1, [[9, 2], [2, 9]])
    after_inv_row_shift2 = self.row_shift(after_inv_mix_columns)
    after_inv_sub_byte2 = self.sub_byte(after_inv_row_shift2, use_inverse=True)
    plaintext = self.round_key_addition(after_inv_sub_byte2, keys[0])

    return plaintext

def encrypt(self, plaintext):
    """多轮加密函数，输入明文，输出密文。key为16的倍数"""
    ciphertext = plaintext
    # 对每一个密钥进行加密操作
    for key in self.keys_list:
        self.key = key
        ciphertext = self._single_encrypt(ciphertext)
    return ciphertext

def decrypt(self, ciphertext):
    """多轮解密函数，输入密文，输出明文。key为16的倍数"""
    plaintext = ciphertext
    # 对每一个密钥进行解密操作（注意逆序操作）
    for key in reversed(self.keys_list):
        self.key = key
        plaintext = self._single_decrypt(plaintext)
    return plaintext

上述的四个函数分别为单轮的加密/解密，以及多轮的加密/解密。其中多轮加密/解密的过程就是将多个密钥进行加密/解密的过程。

我在初始化SAES类的时候，会将输入的密钥进行分割，然后存储在keys_list中，这样就可以在多轮加密/解密的过程中使用了。
```python
    def __init__(self, key,
                 S_BOX=None,
                 S_BOX_INV=None,
                 RCON=None,
                 MixMatrix=None,
                 MixMatrix_INV=None
                 ):
        """ 初始化加密类以及对应的密钥和各种转换盒 """

        ......

        # 检查key的长度是否为16的倍数
        if len(key) % 16 != 0:
            raise ValueError("Key length must be a multiple of 16!")

        # 将key分割为16位长的元素
        self.keys_list = [key[i:i + 16] for i in range(0, len(key), 16)]
```

然后在每一轮中进行for循环加密，对每一个进行加密/解密的密钥进行操作。
```python
    def encrypt(self, plaintext):
        """多轮加密函数，输入明文，输出密文。key为16的倍数"""
        ciphertext = plaintext
        # 对每一个密钥进行加密操作
        for key in self.keys_list:
            self.key = key
            ciphertext = self._single_encrypt(ciphertext)
        return ciphertext

    def decrypt(self, ciphertext):
        """多轮解密函数，输入密文，输出明文。key为16的倍数"""
        plaintext = ciphertext
        # 对每一个密钥进行解密操作（注意逆序操作）
        for key in reversed(self.keys_list):
            self.key = key
            plaintext = self._single_decrypt(plaintext)
        return plaintext
```

下面进行实例化的实现

4.1 双重加密
将S-AES算法通过双重加密进行扩展，分组长度仍然是16 bits，但密钥长度为32 bits。

> 这里的输入16 bits明文和32 bits密钥，输出16 bits密文。密钥长度为32 bits，其中前16 bits为K1，后16 bits为K2。加密过程为先使用K1加密，再使用K2加密，解密过程为先使用K2解密，再使用K1解密。

In [2]:
# 加载实现的加密算法类
from SAES.SAES import SAES

使用S-AES加密算法进行加密/解密处理...


In [3]:
double_key = '10110011100110101101111011001101'
double_saes = SAES(key=double_key)
double_plaintext = '1101011100101100'
print(f'本次SAES加密明文为：{double_plaintext}')
double_encrypted_ciphertext = double_saes.encrypt(double_plaintext)
print(f'通过SAES加密后的密文为：{double_encrypted_ciphertext}')
double_decrypted_plaintext = double_saes.decrypt(double_encrypted_ciphertext)
print(f'通过SAES解密后的明文为：{double_decrypted_plaintext}')

本次SAES加密明文为：1101011100101100
通过SAES加密后的密文为：0110110100111010
通过SAES解密后的明文为：1101011100101100


### 4.2 中间相遇攻击
> 假设你找到了使用相同密钥的明、密文对(一个或多个)，请尝试使用中间相遇攻击的方法找到正确的密钥Key(K1+K2)。

为了进行暴力破解，我们需要遍历所有可能的密钥，然后用每一个可能的密钥进行解密操作，检查解密后的结果是否是预期的明文。

中间相遇攻击适用于块加密算法，其目标是降低暴力破解的复杂性。对于一个双轮SAES，中间相遇攻击的基本思路如下：

- 从明文出发，使用所有可能的第一轮密钥K1进行加密，直到中间状态，并将每一个结果及其对应的K1存储在一个字典中。
- 从密文出发，使用所有可能的第二轮密钥K2进行解密，直到中间状态，并检查每个结果是否在步骤1中创建的字典中。
- 一旦找到匹配项，就可以断定找到了正确的K1和K2。

In [4]:
def meet_in_the_middle_attack(pairs):
    """
    使用中间相遇攻击找到可能的key1和key2对。
    输入：pairs - 明密文对的列表，如[(plaintext1, ciphertext1), (plaintext2, ciphertext2), ...]
    输出：所有可能的(key1, key2)对的列表
    """
    SAES_instance = SAES("0" * 16)  # 创建一个默认密钥的SAES实例
    possible_keys_set = None  # 存储所有明密文对的交集

    for plaintext, ciphertext in pairs:
        encrypt_dict = {}
        current_possible_keys = set()

        # Step 1: 从明文开始加密，直到中间值
        for key1 in range(2**16):  # 遍历所有可能的16位key1
            SAES_instance.key = format(key1, '016b')
            intermediate_ciphertext = SAES_instance._single_encrypt(plaintext)
            encrypt_dict[intermediate_ciphertext] = SAES_instance.key

        # Step 2: 从密文开始解密，直到中间值
        for key2 in range(2**16):  # 遍历所有可能的16位key2
            SAES_instance.key = format(key2, '016b')
            intermediate_plaintext = SAES_instance._single_decrypt(ciphertext)
            if intermediate_plaintext in encrypt_dict:  # 查找是否有匹配的中间值
                current_possible_keys.add((encrypt_dict[intermediate_plaintext], SAES_instance.key))

        if possible_keys_set is None:
            possible_keys_set = current_possible_keys
        else:
            possible_keys_set &= current_possible_keys

    return list(possible_keys_set)

In [5]:
# 1. 只使用一对明密文对
print("1. 只使用一对明密文对")
pairs1 = [("1101011100101100", "0110110100111010")]
potential_keys = meet_in_the_middle_attack(pairs1)

print(f"Number of potential key pairs: {len(potential_keys)}")

1. 只使用一对明密文对
Number of potential key pairs: 41562


可以看到这里的使用单个明密文对，可能的结果会有41562个key对，这个数量是非常大的，所以我们需要更多的明密文对来进行暴力破解。

In [6]:
# 2. 使用两对明密文对
print("2. 使用两对明密文对")
pairs2 = [
    ("1101011100101100", "0110110100111010"),
    ("1111111100000000", "0111011011110111")
]
potential_keys = meet_in_the_middle_attack(pairs2)
print(f"Number of potential key pairs: {len(potential_keys)}")
print(f"Potential key pairs: {potential_keys}")

2. 使用两对明密文对
Number of potential key pairs: 1
Potential key pairs: [('1011001110011010', '1101111011001101')]


可以看当我输入两对明密文对时，可能的结果就只有一个了，这个结果就是正确的密钥。

### 4.3 三重加密
将S-AES算法通过三重加密进行扩展，下面两种模式选择一种完成：
(1)按照32 bits密钥Key(K1+K2)的模式进行三重加密解密，
(2)使用48bits(K1+K2+K3)的模式进行三重加解密。

因为考虑到我们在设计加密类时就已经考虑到了多重加密的情况，所以这里的实现就非常简单了，只需要将密钥长度改为48 bits即可。
也可以参考方案1使用两个key进行加密，但是为了我们代码的连贯性，这里我还是使用了方案2。

In [7]:
triple_key= '101100111001101011011110110011011011101000100111'
print(f'本次SAES加密密钥长度为：{len(triple_key)}')
triple_saes = SAES(key=triple_key)
triple_plaintext = '1101011100101100'
print(f'本次SAES加密明文为：{triple_plaintext}')
triple_encrypted_ciphertext = triple_saes.encrypt(triple_plaintext)
print(f'通过SAES加密后的密文为：{triple_encrypted_ciphertext}')
triple_decrypted_plaintext = triple_saes.decrypt(triple_encrypted_ciphertext)
print(f'通过SAES解密后的明文为：{triple_decrypted_plaintext}')

本次SAES加密密钥长度为：48
本次SAES加密明文为：1101011100101100
通过SAES加密后的密文为：0001100100010001
通过SAES解密后的明文为：1101011100101100


我们这里对加密破解的测试考虑可以验证我们写的加密算法的安全性，同时也完成了双重加密、中间相遇攻击、三重加密的实现。
也可以看到SAES即使使用了双重加密，在有多个明密文对的情况下依然可以在秒级水平被破译。
对加密算法有了更深的理解。